compra Buena

In [7]:
import pandas as pd
import numpy as np
file = 'D:\\Users\\OneDrive\\Escritorio\\DA & BA\\Work\\Tisagar\\Binance\\compra.csv'
df = pd.read_csv(file)

In [8]:
columns_to_drop = [
    'Order Type', 'Fiat Type', 'Exchange rate', 'Maker Fee', 
    'Maker Fee Rate', 'Taker Fee', 'Taker Fee Rate', 
    'Couterparty', 'Status'
]
df.drop(columns=columns_to_drop, inplace=True)
df['Exchange Compra'] = 'Binance - ' + df['Asset Type']
df['Banco Envio'] = 'Lemon'

In [9]:
# Convertir 'Created Time' a formato de fecha
df['Created Time'] = pd.to_datetime(df['Created Time'])
df['Date'] = df['Created Time'].dt.date

# Filtrar datos para cada tipo de activo relevante y calcular el promedio diario de "Price"
asset_types = ['USDT', 'FDUSD', 'USDC', 'DAI']
daily_avgs = {}

for asset in asset_types:
    asset_df = df[df['Asset Type'] == asset]
    daily_avg = asset_df.groupby('Date')['Price'].mean().reset_index()
    daily_avg.columns = ['Date', f'Avg_{asset}_Price']
    daily_avgs[asset] = daily_avg

# Unir los promedios diarios al DataFrame original
for asset, avg_df in daily_avgs.items():
    df = pd.merge(df, avg_df, on='Date', how='left')

# Crear una columna para el promedio diario combinado
df['Avg_Daily_Price'] = df['Avg_USDT_Price']

# Usar los promedios de FDUSD, USDC y DAI si no hay promedio de USDT
for asset in asset_types[1:]:
    df['Avg_Daily_Price'].fillna(df[f'Avg_{asset}_Price'], inplace=True)

# Filtrar los tipos de activos que deben ser actualizados
assets_to_update = ['BTC', 'ETH', 'BNB']

# Actualizar "Price" y "Quantity" para "Asset Type" = BTC, ETH o BNB
df.loc[df['Asset Type'].isin(assets_to_update), 'Price'] = df['Avg_Daily_Price']
df.loc[df['Asset Type'].isin(assets_to_update), 'Quantity'] = df['Total Price'] / df['Price']

In [10]:
column_order = ['Created Time','Exchange Compra', 'Total Price', 'Price', 'Quantity', 'Banco Envio', 'Order Number']
df = df[column_order]

In [11]:
if isinstance(df['Created Time'].iloc[0], str):
    df['Created Time'] = pd.to_datetime(df['Created Time'])
new_rows = []
last_date = None

for index, row in df.iterrows():
    current_date = row['Created Time'].date()  # Obtener la fecha (solo fecha, sin hora)
    
    if last_date is None or current_date != last_date:
        new_rows.append(pd.Series([None] * len(df.columns), index=df.columns))  # Agregar fila en blanco
        last_date = current_date
    
    new_rows.append(row)  # Agregar la fila actual
    
# Crear un nuevo DataFrame con las filas agregadas
df_n = pd.DataFrame(new_rows)

In [12]:
df_n.to_excel('BinanceCompra_limpio.xlsx', index=False)